In [1]:
import pandas as pd
import numpy as np
import torch
import random

# Additional PyTorch-specific imports if needed

import torch
import torch.nn as nn
import torch.nn.functional as F


import torch.optim as optim

# Generator Loader

In [ ]:


class gen_Data_loader():
  def __init__(self,batch_size):
    self.batch_size = batch_size
    self.token_stream = []

  def create_batches(self,data_file):
    self.token_stream = []
    with open(data_file, "r") as f:
      for line in f:
        line = line.strip()
        line = line.split()
        parse_line = [int(x) for x in line]
        if len(parse_line) == 20:

          self.token_stream.append(parse_line)
    self.num_batch = int(len(self.token_stream) / self.batch_size)
    self.token_stream = self.token_stream[:self.num_batch * self.batch_size]
    self.sequence_batch = np.split(np.array(self.token_stream), self.num_batch, 0)
    self.pointer = 0

  def next_batch(self):
    ret = self.sequence_batch[self.pointer]
    self.pointer = (self.pointer +1) % self.num_batch
    return ret
  def reset_pointer(self):
    self.pointer = 0





# Discriminator Loader

In [ ]:



class Dis_dataloader():
  def __init__(self, batch_size):
    self.batch_size = batch_size
    self.sentences = np.array([])
    self.labels = np.array([])

  def load_train_data(self, positive_file, negative_file):
    positive_examples = []
    negative_examples = []
    with open(positive_file) as fin:
      for line in fin:
          line = line.strip()
          line = line.split()
          parse_line = [int(x) for x in line]
          positive_examples.append(parse_line)
    with open(negative_file)as fin:
      for line in fin:
          line = line.strip()
          line = line.split()
          parse_line = [int(x) for x in line]
          if len(parse_line) == 20:
            negative_examples.append(parse_line)
          self.sentences = np.array(positive_examples + negative_examples)
          positive_labels = [[0, 1] for _ in positive_examples]
          negative_labels = [[1, 0] for _ in negative_examples]
          self.labels = np.concatenate([positive_labels, negative_labels], 0)

          # Shuffle the data
          shuffle_indices = np.random.permutation(np.arange(len(self.labels)))
          self.sentences = self.sentences[shuffle_indices]
          self.labels = self.labels[shuffle_indices]

          # Split batches
          self.num_batch = int(len(self.labels) / self.batch_size)
          self.sentences = self.sentences[:self.num_batch * self.batch_size]
          self.labels = self.labels[:self.num_batch * self.batch_size]
          self.sentences_batches = np.split(self.sentences, self.num_batch, 0)
          self.labels_batches = np.split(self.labels, self.num_batch, 0)

          self.pointer = 0
  def next_batch(self):
      ret = self.sentences_batches[self.pointer], self.labels_batches[self.pointer]
      self.pointer = (self.pointer + 1) % self.num_batch
      return ret

  def reset_pointer(self):
      self.pointer = 0




# Discriminator Code 

In [ ]:

class Highway(nn.Module):
    def __init__(self, size, num_layers=1, bias=-2.0, activation=F.relu):
        super().__init__()
        self.num_layers = num_layers
        self.bias = bias
        self.activation = activation
        # one linear and one gate per layer
        self.linears = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.gates   = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])

    def forward(self, x):
        # x: [batch, size]
        for lin, gate in zip(self.linears, self.gates):
            g = self.activation(lin(x))
            t = torch.sigmoid(gate(x) + self.bias)
            x = t * g + (1 - t) * x
        return x



#Coding the actual discriminator itself

class Discriminator(nn.Module):
    def __init__(self, sequence_length, data_size, l2_reg_lambda=0.0):
        """
        Args:
          sequence_length: int, length of your input “sequence” dimension
          data_size:        int, the number of channels/features per step
          l2_reg_lambda:    float, coefficient for L2 penalty (pass as weight_decay to your optimizer)
        """
        super().__init__()
        self.l2_reg_lambda = l2_reg_lambda

        # two 2D conv layers (assumes input x shaped [B, 1, L, D])
        self.conv1 = nn.Conv2d(1,  64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)

        # compute flattened size after conv2
        flat_size = 32 * sequence_length * data_size

        # highway block
        self.highway = Highway(flat_size, num_layers=2)

        # six fully-connected layers
        self.fc1 = nn.Linear(flat_size, 1024)
        self.fc2 = nn.Linear(1024,     512)
        self.fc3 = nn.Linear(512,      256)
        self.fc4 = nn.Linear(256,      128)
        self.fc5 = nn.Linear(128,       64)
        self.fc6 = nn.Linear(64,        32)

        # a 1×1 conv + pool on final FC output
        self.conv_pool_conv = nn.Conv2d(32, 64, kernel_size=1)
        self.pool            = nn.MaxPool2d(kernel_size=1)

        # final score layer
        self.output = nn.Linear(64, 2)

    def forward(self, x, dropout_keep_prob=0.5):
        """
        Args:
          x:                  Tensor of shape [batch, 1, sequence_length, data_size]
          dropout_keep_prob:  float in (0,1], the keep probability
        Returns:
          scores:      [batch, 2]
          predictions: [batch] long tensor of predicted class
        """
        # conv stack
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        # flatten
        x = x.view(x.size(0), -1)

        # highway + dropout
        x = self.highway(x)
        x = F.dropout(x, p=1-dropout_keep_prob, training=self.training)

        # six FC + dropout blocks
        for fc in (self.fc1, self.fc2, self.fc3, self.fc4, self.fc5, self.fc6):
            x = F.relu(fc(x))
            x = F.dropout(x, p=1-dropout_keep_prob, training=self.training)

        # reshape for conv-pool
        x = x.unsqueeze(-1).unsqueeze(-1)    # → [B, 32, 1, 1]
        x = F.relu(self.conv_pool_conv(x))   # → [B, 64, 1, 1]
        x = self.pool(x)                     # → [B, 64, 1, 1]
        x = x.view(x.size(0), -1)            # → [B, 64]

        # final scores & preds
        scores      = self.output(x)         # [B, 2]
        predictions = torch.argmax(scores, dim=1)

        return scores, predictions




# Generator Code

In [ ]:
class Generator(nn.Module):
    def __init__(self,
                 hidden_dim: int,
                 seq_len: int,
                 start_value: float,
                 reward_gamma: float = 0.95,
                 temperature: float = 1.0,
                 grad_clip: float = 5.0):
        """
        Args:
            hidden_dim:    hidden size of the LSTM cell
            seq_len:       length of sequence to generate
            start_value:   initial value to feed in at t=0 when sampling
            reward_gamma:  discount factor (if you’ll implement RL later)
            temperature:   scaling factor for sampling
            grad_clip:     max‐norm for gradient clipping
        """
        super().__init__()
        self.hidden_dim   = hidden_dim
        self.seq_len      = seq_len
        self.start_value  = start_value
        self.reward_gamma = reward_gamma
        self.temperature  = temperature
        self.grad_clip    = grad_clip

        # your “recurrent unit” → LSTMCell
        self.lstm_cell = nn.LSTMCell(1, hidden_dim)

        # your “output unit” → project hidden to numerical values
        self.output_linear = nn.Linear(hidden_dim, 1)

    def forward(self, x: torch.Tensor):
        """
        Teacher-forcing pass for pretraining.
        Args:
            x: [batch, seq_len, 1] tensor of numerical values
        Returns:
            outputs: [batch, seq_len, 1]
        """
        batch_size = x.size(0)
        device     = x.device

        # initialize (h,c) to zeros
        h = torch.zeros(batch_size, self.hidden_dim, device=device)
        c = torch.zeros(batch_size, self.hidden_dim, device=device)

        outputs_time = []
        for t in range(self.seq_len):
            h, c = self.lstm_cell(x[:, t, :], (h, c))
            output = self.output_linear(h).unsqueeze(1)  # [B, 1]
            outputs_time.append(output)

        # [B, T, 1]
        outputs = torch.cat(outputs_time, dim=1)
        return outputs

    def sample(self, batch_size: int = None):
        """
        Sequential sampling (for adversarial roll-outs).
        Args:
            batch_size: if None, uses the batch_size given in forward calls
        Returns:
            samples: [batch_size, seq_len, 1] tensor of sampled numerical values
        """
        if batch_size is None:
            raise ValueError("Please pass batch_size when sampling")

        device = next(self.parameters()).device
        h = torch.zeros(batch_size, self.hidden_dim, device=device)
        c = torch.zeros(batch_size, self.hidden_dim, device=device)

        # start with the start value
        inp = torch.full((batch_size, 1), self.start_value, dtype=torch.float, device=device)
        samples = []

        for _ in range(self.seq_len):
            h, c = self.lstm_cell(inp, (h, c))    # [B, H]
            output = self.output_linear(h) / self.temperature
            inp = output                          # feed output as next input
            samples.append(output.unsqueeze(1))

        # [B, T, 1]
        return torch.cat(samples, dim=1)

    def pretrain_loss(self, outputs: torch.Tensor, target: torch.Tensor):
        """
        MSE loss for numerical values.
        Args:
            outputs: [B, T, 1]
            target: [B, T, 1]
        """
        mse = F.mse_loss(outputs, target, reduction='mean')
        return mse
    def pretrain_step(self, optimizer, outputs: torch.Tensor, target: torch.Tensor):
        """
        Perform a pretraining step with gradient clipping.
        Args:
            optimizer: PyTorch optimizer for the generator.
            outputs: [B, T, 1] tensor of generated numerical values.
            target: [B, T, 1] tensor of target numerical values.
        Returns:
            loss: Scalar loss value.
        """
        # Compute the MSE loss
        loss = F.mse_loss(outputs, target, reduction='mean')

        # Zero the gradients
        optimizer.zero_grad()

        # Backpropagate the loss
        loss.backward()

        # Clip gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(self.parameters(), self.grad_clip)

        # Update the model parameters
        optimizer.step()

        return loss


# Rollout Implementation

In [ ]:

class Rollout(nn.Module):
    def __init__(self, generator, update_rate):
        """
        generator: Generator model with LSTM layer
        update_rate: float in (0,1), how fast rollout net tracks generator
        """
        super().__init__()
        self.generator = generator
        self.update_rate = update_rate
        self.sequence_length = generator.sequence_length
        self.device = next(generator.parameters()).device
        
        # Create LSTM and linear layers matching generator's architecture
        self.lstm = nn.LSTM(
            input_size=generator.lstm.input_size,
            hidden_size=generator.lstm.hidden_size,
            num_layers=generator.lstm.num_layers,
            batch_first=True
        )
        
        self.output_layer = nn.Linear(
            generator.lstm.hidden_size,
            generator.output_layer.out_features
        )
        
        # Initialize with generator's weights
        self.update_params()

    def _rollout(self, prefix, given_len):
        """Generate continuation of the prefix sequence"""
        batch_size = prefix.size(0)
        
        # Process prefix through LSTM
        hidden = None
        output, hidden = self.lstm(prefix[:, :given_len, :], hidden)
        
        # Generate remaining sequence
        current = prefix[:, given_len-1:given_len, :]
        samples = [prefix[:, :given_len, :]]
        
        for t in range(given_len, self.sequence_length):
            output, hidden = self.lstm(current, hidden)
            next_value = self.output_layer(output)
            samples.append(next_value)
            current = next_value
            
        return torch.cat(samples, dim=1)

    def get_reward(self, input_x, rollout_num, discriminator):
        """Calculate rewards for each position in the sequence"""
        batch_size = input_x.size(0)
        rewards = []
        
        for n in range(rollout_num):
            for given_len in range(1, self.sequence_length):
                samples = self._rollout(input_x, given_len)
                with torch.no_grad():
                    scores = discriminator(samples)
                    prob = torch.sigmoid(scores)
                if n == 0:
                    rewards.append(prob.cpu().numpy())
                else:
                    rewards[given_len-1] += prob.cpu().numpy()

            # Reward for complete sequence
            with torch.no_grad():
                scores = discriminator(input_x)
                prob = torch.sigmoid(scores)
            if n == 0:
                rewards.append(prob.cpu().numpy())
            else:
                rewards[self.sequence_length-1] += prob.cpu().numpy()

        rewards = np.array(rewards).T / float(rollout_num)
        return rewards

    def update_params(self):
        """Update rollout network weights based on generator"""
        with torch.no_grad():
            for rollout_param, gen_param in zip(self.lstm.parameters(), 
                                              self.generator.lstm.parameters()):
                rollout_param.data.copy_(
                    self.update_rate * rollout_param.data + 
                    (1 - self.update_rate) * gen_param.data
                )
            
            for rollout_param, gen_param in zip(self.output_layer.parameters(), 
                                              self.generator.output_layer.parameters()):
                rollout_param.data.copy_(
                    self.update_rate * rollout_param.data + 
                    (1 - self.update_rate) * gen_param.data
                )


# Generator Hyperparameters

In [ ]:

HIDDEN_DIM = 32 # hidden state dimension of lstm cell
SEQ_LENGTH = 20 # sequence length
START_TOKEN = 0
PRE_EPOCH_NUM = 120 # supervise (maximum likelihood estimation) epochs
SEED = 88
BATCH_SIZE = 64





# Discriminator Hyperparameters

In [ ]:


dis_dropout_keep_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64


# General Hyperparameters and Directories 

In [ ]:


TOTAL_BATCH = 200
positive_file = 'save/real_data.txt'
negative_file = 'save/generator_sample.txt'
eval_file = 'save/eval_file.txt'
generated_num = 10000


# Main Model Code 

In [ ]:


def generate_samples(model, batch_size, generated_num, output_file):
  model.eval()
  generated_samples = []

  for _ in range(int(generated_num / batch_size)):
    samples = model.generate(batch_size)
    generated_samples.extend(samples.cpu().numpy())

  with open(output_file, 'w') as fout:
    for sequence in generated_samples:
      buffer = ','.join([str(x) for x in sequence]) + '\n'
      fout.write(buffer)

def target_loss(model, data_loader, criterion):
  model.eval()
  nll = []

  for batch in data_loader:
    batch = batch.to(device)
    g_loss = criterion(model(batch), batch)
    nll.append(g_loss.item())

  return np.mean(nll)

def pre_train_epoch(model, data_loader, optimizer, criterion):
  model.train()
  supervised_g_losses = []

  for batch in data_loader:
    batch = batch.to(device)
    optimizer.zero_grad()
    output = model(batch)
    g_loss = criterion(output, batch)
    g_loss.backward()
    optimizer.step()
    supervised_g_losses.append(g_loss.item())

  return np.mean(supervised_g_losses)


def main():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)

  gen_data_loader = gen_Data_loader(BATCH_SIZE)
  likelihood_data_loader = gen_Data_loader(BATCH_SIZE)
  dis_data_loader = Dis_dataloader(BATCH_SIZE)

  generator = Generator(vocab_size=1000, embedding_dim=50, hidden_dim=21, sequence_length=21, start_token=START_TOKEN).to(device)
  discriminator = Discriminator(sequence_length=20, num_classes=2, embedding_dim=50, hidden_dim=64, dropout_prob=0.4).to(device)

  gen_optimizer = optim.Adam(generator.parameters(), lr=1e-3)
  dis_optimizer = optim.Adam(discriminator.parameters(), lr=1e-3)
  criterion = nn.CrossEntropyLoss()

  # Pre-training Generator
  print("Pre Training Generator")
  for epoch in range(PRE_EPOCH_NUM):
    loss = pre_train_epoch(generator, gen_data_loader, gen_optimizer, criterion)
    if epoch % 5 == 0:
      generate_samples(generator, BATCH_SIZE, generated_num, eval_file)
      likelihood_data_loader.create_batches(eval_file)
      test_loss = target_loss(generator, likelihood_data_loader, criterion)
      print(f"Epoch: {epoch}, NLL: {test_loss}")

  # Pre-training Discriminator
  print("Pre Training Discriminator")
  for _ in range(50):
    generate_samples(generator, BATCH_SIZE, generated_num, negative_file)
    dis_data_loader.load_train_data(positive_file, negative_file)

    for _ in range(3):
      for x_batch, y_batch in dis_data_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        dis_optimizer.zero_grad()
        logits = discriminator(x_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        dis_optimizer.step()

  # Adversarial Training
  rollout = Rollout(generator, 0.8)
  for total_batch in range(TOTAL_BATCH):
    for _ in range(1):
      samples = generator.generate(BATCH_SIZE)
      rewards = rollout.get_reward(samples, discriminator)
      gen_optimizer.zero_grad()
      loss = -torch.mean(rewards)
      loss.backward()
      gen_optimizer.step()

    rollout.update_params()

    for _ in range(5):
      generate_samples(generator, BATCH_SIZE, generated_num, negative_file)
      dis_data_loader.load_train_data(positive_file, negative_file)

      for _ in range(3):
        for x_batch, y_batch in dis_data_loader:
          x_batch, y_batch = x_batch.to(device), y_batch.to(device)
          dis_optimizer.zero_grad()
          logits = discriminator(x_batch)
          loss = criterion(logits, y_batch)
          loss.backward()
          dis_optimizer.step()

if __name__ == '__main__':
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  main()









